# Install / Import / Config

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import os
from pathlib import Path
from dotenv import load_dotenv
import edurel.utils.llm as llmu
import edurel.utils.duckdb as ddbu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases/"

# Database

In [29]:
con = ddbu.file_con(DB_DIR + "db-ccfraud")
schema = ddbu.schema(con)

In [30]:
print(ddbu.schema(con))
# con.close()

Table: ccfraud (userid SMALLINT NULL, card SMALLINT NULL, ts TIMESTAMP NULL, amount FLOAT NULL, use_chip VARCHAR NULL, mname BIGINT NULL, mcity VARCHAR NULL, mstate VARCHAR NULL, zip VARCHAR NULL, mcc INTEGER NULL, err1 VARCHAR NULL, err2 VARCHAR NULL, err3 VARCHAR NULL, is_fraud VARCHAR NULL)



## Sample Data

In [31]:
sql = """
select * from ccfraud limit 11;
"""
ddbu.sql_print(con, sql)

┌────────┬───────┬─────────────────────┬────────┬────────────────────┬──────────────────────┬───────────────┬─────────┬─────────┬───────┬─────────┬─────────┬─────────┬──────────┐
│ userid │ card  │         ts          │ amount │      use_chip      │        mname         │     mcity     │ mstate  │   zip   │  mcc  │  err1   │  err2   │  err3   │ is_fraud │
│ int16  │ int16 │      timestamp      │ float  │      varchar       │        int64         │    varchar    │ varchar │ varchar │ int32 │ varchar │ varchar │ varchar │ varchar  │
├────────┼───────┼─────────────────────┼────────┼────────────────────┼──────────────────────┼───────────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┼──────────┤
│      0 │     0 │ 2010-01-02 16:41:00 │  26.32 │ Swipe Transaction  │   838425044734233142 │ Mira Loma     │ CA      │ 91752   │  4814 │ NULL    │ NULL    │ NULL    │ No       │
│      0 │     0 │ 2010-01-03 06:54:00 │  57.82 │ Swipe Transaction  │   -34551508091458520 │ La Verne   

# Queries

## q1a

In [32]:
q1a = """
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;
"""

## q1b

In [33]:
q1b = """
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
the output should be presented as follows:
- format('{:5.2f}%', frac) for column frac
sorted by tyear;
"""

## q1 solution

In [34]:
sql = """
SELECT 
    EXTRACT(YEAR FROM ts) AS year,
    COUNT(*) AS noof_year,
    (SELECT COUNT(*) FROM ccfraud) AS noof_all,
    FORMAT('{:5.2f}%', COUNT(*) * 100.0 / (SELECT COUNT(*) FROM ccfraud)) AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY year;
"""
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────┐
│ year  │ noof_year │ noof_all │  frac   │
│ int64 │   int64   │  int64   │ varchar │
├───────┼───────────┼──────────┼─────────┤
│  2010 │   1491225 │ 16575073 │  9.00%  │
│  2011 │   1570551 │ 16575073 │  9.48%  │
│  2012 │   1610829 │ 16575073 │  9.72%  │
│  2013 │   1650917 │ 16575073 │  9.96%  │
│  2014 │   1672343 │ 16575073 │ 10.09%  │
│  2015 │   1701371 │ 16575073 │ 10.26%  │
│  2016 │   1708924 │ 16575073 │ 10.31%  │
│  2017 │   1723360 │ 16575073 │ 10.40%  │
│  2018 │   1721615 │ 16575073 │ 10.39%  │
│  2019 │   1723938 │ 16575073 │ 10.40%  │
├───────┴───────────┴──────────┴─────────┤
│ 10 rows                      4 columns │
└────────────────────────────────────────┘



## q2a

In [35]:
q2a = """
create a SQL query that outputs the following columns:
- trx_time_delta_bin time difference between current and previous transaction per userid either 'less_5min' or 'more_eq_5min'
- noof_trx number of records per bin
- noof_fraud number of fraud records per bin
- frac fraction of fraud records per bin
sorted by userid, trx_time_delta_bin;
"""

## q2b

In [36]:
q2b = """
create a SQL query that outputs the following columns:
- trx_time_delta_bin time difference between current and previous transaction per userid either 'less_5min' or 'more_eq_5min'
- noof_trx number of records per bin
- noof_fraud number of fraud records per bin
- frac fraction of fraud records per bin
regard the following:
- only consider bins where noof_fraud > 0
sorted by userid, trx_time_delta_bin;
"""

## q2 solution

In [37]:
sql = """
with
  base1 as (
    select
      userid,
      ts,
      lag(ts) over (partition by userid order by ts) as ts_p,
      mcity,
      lag(mcity) over (partition by userid order by ts) as mcity_p,
      is_fraud
    from ccfraud
    -- where ts >= to_date('2019-01-01', 'YYYY-MM-DD')
  ),
  base2 as (
    select
      userid,
      ts,
      ts_p,
      mcity,
      mcity_p,
      (extract(epoch from ts) - extract(epoch from ts_p)) / 60 as abstand_minuten,
      is_fraud
    from base1
    where not ts_p is null
  ),
  base as (
    select
      case
        when abstand_minuten < 5 then 'a) bis 5 min'
        else 'b) ab 5 min'
      end as abstand_minuten,
      is_fraud
    from base2
    where mcity<>mcity_p
  ),
  proportion(abstand_minuten, anz_trx, ant_ok, ant_fraud) as (
    select
      abstand_minuten,
      count(*),
      sum(case when is_fraud='No' then 1 else 0 end),
      sum(case when is_fraud='Yes' then 1 else 0 end)
    from base
    group by abstand_minuten
  ),
  ratio(abstand_minuten, anz_trx, ant_ok, ant_fraud, quote_ok, quote_fraud) as (
    select
      abstand_minuten,
      anz_trx,
      ant_ok,
      ant_fraud,
      round(ant_ok*100/anz_trx::numeric, 4),
      round(ant_fraud*100/anz_trx::numeric, 4)
     from proportion
  )
select
  abstand_minuten as "abstand_minuten",
  anz_trx as "anz_trx",
  ant_ok as "ant_ok",
  ant_fraud as "ant_fraud",
  quote_ok as "quote_ok",
  quote_fraud as "quote_fraud"
from ratio
order by abstand_minuten;

"""
ddbu.sql_print(con, sql)

┌─────────────────┬─────────┬─────────┬───────────┬──────────┬─────────────┐
│ abstand_minuten │ anz_trx │ ant_ok  │ ant_fraud │ quote_ok │ quote_fraud │
│     varchar     │  int64  │ int128  │  int128   │  double  │   double    │
├─────────────────┼─────────┼─────────┼───────────┼──────────┼─────────────┤
│ a) bis 5 min    │  118522 │  118041 │       481 │  99.5942 │      0.4058 │
│ b) ab 5 min     │ 7709523 │ 7698781 │     10742 │  99.8607 │      0.1393 │
└─────────────────┴─────────┴─────────┴───────────┴──────────┴─────────────┘



## q4

In [38]:
q4 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name, gehalt, bonus aus tabelle mitarbeiter
nur mitarbeiter ohne bonus
sortiert nach name
"""

## q5

In [39]:
q5 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name, gehalt, bonus aus tabelle mitarbeiter
nur mitarbeiter mit bonus
sortiert nach name
"""

## q6

In [40]:
q6 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid, name, gehalt, bonus aus tabelle mitarbeiter
- gesamteinkommen pro mitarbeiter 
nur für abteilung 17
sortiert nach mid
"""

## q7

In [41]:
q7 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid, name, gehalt aus tabelle mitarbeiter
- gehaltsklasse pro mitarbeiter 
es gibt folgende gehaltsklassen:
- niedrig <= 35000
- mittel > 35000
- hoch > 100000
sortiert nach gehalt
"""

## q8

In [42]:
q8 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- bezeichung aus tabelle orgeinheit mit alias orgeinheit
- mid, name aus tabelle mitarbeiter
sortiert nach orgeinheit, mid
"""

## q9

In [43]:
q9 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- titel aus tabelle projekt mit alias projekt
- mid, name aus tabelle mitarbeiter
sortiert nach titel, mid
"""

## q10

In [44]:
q10 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- bezeichnung mit alias obereinheit aus tabelle orgeinheit
- bezeichnung mit alias untereinheit aus tabelle orgeinheit
untereinheit soll direkte untereinheit von obereinheit sein
sortiert nach obereinheit, untereinheit
"""

## q11

In [45]:
q11 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid aus tabelle mitarbeiter
- name mit alias abteilungsleiter aus tabelle mitarbeiter
- bezeichnung mit alias orgeinheit aus tabelle orgeinheit
es sollen nur abteilungsleiter ausgegeben werden
sortiert nach mid
"""

## q12

In [46]:
q12 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid aus tabelle mitarbeiter
- name mit alias mitarbeiter aus tabelle mitarbeiter
es sollen nur mitarbeiter ausgegeben werden, die keine abteilungsleiter sind
sortiert nach mid
"""

## q13

In [47]:
q13 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name mit alias mitarbeiter aus tabelle mitarbeiter
- gehalt des mitarbeiters mit alias mgehalt aus tabelle mitarbeiter
- name mit alias chef aus tabelle mitarbeiter
- gehalt des chefs mit alias cgehalt aus tabelle mitarbeiter
chef ist direkte vorgesetzte von mitarbeiter
mitarbeiter verdient mehr als chef
sortiert nach mitarbeiter
"""

## q14

In [48]:
q14 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- max_gehalt
max_gehalt soll das maximale gehalt aller mitarbeiter sein
"""

## q15

In [49]:
q15 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- bezeichnung mit alias abteilung aus tabelle orgeinheit
- max_gehalt_abt
max_gehalt_abt soll das maximale gehalt aller mitarbeiter pro abteilung sein
sortiert nach absteigendem max_gehalt_abt
"""

## q16

In [50]:
q16 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- eintrittsjahr berechnet aus eintrittsdatum
- gehalt_jahr soll das gesamte gehalt aller mitarbeiter pro eintrittsjahr sein
sortiert nach eintrittsjahr
"""

## q17

In [51]:
q17 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- jahrzehnt berechnet aus eintrittsdatum
- gehalt_jahrzehnt soll das gesamte gehalt aller mitarbeiter pro jahrzehnt sein
sortiert nach jahrzehnt
"""

## q18

In [52]:
q18 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- bezeichnung mit alias orgeinheit aus tabelle orgeinheit
- jahrzehnt berechnet aus eintrittsdatum, nutze floor für die berechnung
- gehalt_jahrzehnt soll das gesamte gehalt aller mitarbeiter pro orgeinheit und jahrzehnt sein
sortiert nach oeid, jahrzehnt
"""

## q19

In [53]:
q19 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name, gehalt aus tabelle mitarbeiter
nur mitarbeiter, die mehr als der durchschnitt aller mitarbeiter verdienen
sortiert nach mid
"""

## q20

In [54]:
q20 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid, name, gehalt aus tabelle mitarbeiter
- durchschnittsgehalt aller mitarbeiter, alias dgehalt
- diff zum durchschnittsgehalt, alias diff_dgehalt
nur mitarbeiter, die mehr als der durchschnitt aller mitarbeiter verdienen
nutze cte für die berechnung
runde berechnung auf 2 nachkommastellen
sortiert nach mid
"""

## q21

In [55]:
q21 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- bezeichnung mit alias orgeinheit aus tabelle orgeinheit
- mid, name, gehalt aus tabelle mitarbeiter
- durchschnittsgehalt aller mitarbeiter in der gleichen abteilung, alias dgehalt_abt
- diff zum durchschnittsgehalt, alias diff_dgehalt_abt
nutze cte für die berechnung
runde berechnung auf 2 nachkommastellen
sortiert nach oeid, mid
"""

## q22

In [56]:
q22 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- mid, name aus tabelle mitarbeiter
nur mitarbeiter die in einer abteilung arbeiten, die von 'Meier' geleitet wird
sortiert nach mid
"""

## q23

In [57]:
q23 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- oeid, bezeichnung mit alias orgeinheit aus tabelle orgeinheit
- anzahl mitarbeiter in der orgeinheit, alias anzahl_mitarbeiter
- nur orgeinheiten mit den wenigsten mitarbeitern
sortiert nach oeid
"""

## q24

In [58]:
q24 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- oeid, bezeichnung mit alias orgeinheit aus tabelle orgeinheit
- durchschnittsgehalt aller mitarbeiter in der orgeinheit, alias dgehalt
nur orgeinheiten maximalen durchschnittsgehalt
sortiert nach oeid
"""

# ARCTICTEXT2SQL

## q1a

In [59]:
# arctic = llmu.ollama_c(llmu.ARCTICTEXT2SQL)

In [60]:
# q = q1a
# print(q)
# sql = llmu.chat_text_to_sql(arctic, schema, q)
# print(sql)

In [61]:
sql = """
SELECT 
    strftime('%Y', ts) AS tyear,
    COUNT(*) AS noof_year,
    (SELECT COUNT(*) FROM ccfraud) AS noof_all,
    CAST(COUNT(*) AS REAL) / (SELECT COUNT(*) FROM ccfraud) AS frac
FROM 
    ccfraud
GROUP BY 
    strftime('%Y', ts)
ORDER BY 
    tyear;
"""

In [62]:
# ddbu.sql_print(con, sql)

# GLM46

In [63]:
glm46 = llmu.stats_c(llmu.GLM46)

## q1a

In [64]:
q = q1a
print(q)
sql = llmu.chat_text_to_sql(glm46, schema, q)
print(sql)


create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;


SELECT 
    EXTRACT(YEAR FROM ts) AS year,
    COUNT(*) AS noof_year,
    (SELECT COUNT(*) FROM ccfraud) AS noof_all,
    COUNT(*)::FLOAT / (SELECT COUNT(*) FROM ccfraud) AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY year;



In [65]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────────┐
│ year  │ noof_year │ noof_all │    frac     │
│ int64 │   int64   │  int64   │    float    │
├───────┼───────────┼──────────┼─────────────┤
│  2010 │   1491225 │ 16575073 │  0.08996793 │
│  2011 │   1570551 │ 16575073 │ 0.094753794 │
│  2012 │   1610829 │ 16575073 │  0.09718382 │
│  2013 │   1650917 │ 16575073 │ 0.099602394 │
│  2014 │   1672343 │ 16575073 │  0.10089506 │
│  2015 │   1701371 │ 16575073 │ 0.102646366 │
│  2016 │   1708924 │ 16575073 │  0.10310205 │
│  2017 │   1723360 │ 16575073 │ 0.103972994 │
│  2018 │   1721615 │ 16575073 │  0.10386772 │
│  2019 │   1723938 │ 16575073 │  0.10400787 │
├───────┴───────────┴──────────┴─────────────┤
│ 10 rows                          4 columns │
└────────────────────────────────────────────┘



## q1b

In [66]:
q = q1b
print(q)
sql = llmu.chat_text_to_sql(glm46, schema, q)
print(sql)


create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
the output should be presented as follows:
- format('{:5.2f}%', frac) for column frac
sorted by tyear;


SELECT 
    EXTRACT(YEAR FROM ts) AS year,
    COUNT(*) AS noof_year,
    SUM(COUNT(*)) OVER () AS noof_all,
    FORMAT('%5.2f%%', (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ())) AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY year;



## q2a

In [67]:
q = q2a
print(q)
sql = llmu.chat_text_to_sql(glm46, schema, q)
print(sql)


create a SQL query that outputs the following columns:
- trx_time_delta_bin time difference between current and previous transaction per userid either 'less_5min' or 'more_eq_5min'
- noof_trx number of records per bin
- noof_fraud number of fraud records per bin
- frac fraction of fraud records per bin
sorted by userid, trx_time_delta_bin;


WITH time_diffs AS (
    SELECT 
        userid,
        ts,
        is_fraud,
        ts - LAG(ts) OVER (PARTITION BY userid ORDER BY ts) AS time_diff
    FROM ccfraud
),
binned_data AS (
    SELECT 
        userid,
        is_fraud,
        CASE 
            WHEN time_diff < INTERVAL '5 minutes' THEN 'less_5min' 
            ELSE 'more_eq_5min' 
        END AS trx_time_delta_bin
    FROM time_diffs
    WHERE time_diff IS NOT NULL
)
SELECT 
    trx_time_delta_bin,
    COUNT(*) AS noof_trx,
    SUM(CASE WHEN is_fraud = 'Yes' THEN 1 ELSE 0 END) AS noof_fraud,
    ROUND(SUM(CASE WHEN is_fraud = 'Yes' THEN 1 ELSE 0 END)::NUMERIC / COUNT(*) * 100, 2) 

In [68]:
ddbu.sql_print(con, sql)

┌────────────────────┬──────────┬────────────┬────────┐
│ trx_time_delta_bin │ noof_trx │ noof_fraud │  frac  │
│      varchar       │  int64   │   int128   │ double │
├────────────────────┼──────────┼────────────┼────────┤
│ less_5min          │      510 │          0 │    0.0 │
│ more_eq_5min       │    11051 │         17 │   0.15 │
│ less_5min          │      214 │          1 │   0.47 │
│ more_eq_5min       │     5446 │         29 │   0.53 │
│ less_5min          │      777 │          6 │   0.77 │
│ more_eq_5min       │    25520 │         34 │   0.13 │
│ less_5min          │      401 │          2 │    0.5 │
│ more_eq_5min       │     8421 │         18 │   0.21 │
│ less_5min          │      191 │          0 │    0.0 │
│ more_eq_5min       │     9183 │          1 │   0.01 │
│     ·              │       ·  │          · │     ·  │
│     ·              │       ·  │          · │     ·  │
│     ·              │       ·  │          · │     ·  │
│ less_5min          │      562 │          0 │  

# GEMINI3PRO

In [16]:
gemini3pro = llmu.stats_c(llmu.GEMINI3PRO)

## q1a

In [ ]:
q = q1a
print(q)
sql = llmu.chat_text_to_sql(gemini3pro, schema, q)
print(sql)

q1:
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;


SELECT 
    EXTRACT(YEAR FROM ts) AS year, 
    COUNT(*) AS noof_year, 
    (SELECT COUNT(*) FROM ccfraud) AS noof_all, 
    COUNT(*)::FLOAT / (SELECT COUNT(*) FROM ccfraud) AS frac 
FROM ccfraud 
GROUP BY year 
ORDER BY year;



In [32]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────────┐
│ year  │ noof_year │ noof_all │    frac     │
│ int64 │   int64   │  int64   │    float    │
├───────┼───────────┼──────────┼─────────────┤
│  2010 │   1491225 │ 16575073 │  0.08996793 │
│  2011 │   1570551 │ 16575073 │ 0.094753794 │
│  2012 │   1610829 │ 16575073 │  0.09718382 │
│  2013 │   1650917 │ 16575073 │ 0.099602394 │
│  2014 │   1672343 │ 16575073 │  0.10089506 │
│  2015 │   1701371 │ 16575073 │ 0.102646366 │
│  2016 │   1708924 │ 16575073 │  0.10310205 │
│  2017 │   1723360 │ 16575073 │ 0.103972994 │
│  2018 │   1721615 │ 16575073 │  0.10386772 │
│  2019 │   1723938 │ 16575073 │  0.10400787 │
├───────┴───────────┴──────────┴─────────────┤
│ 10 rows                          4 columns │
└────────────────────────────────────────────┘



## q1b

In [ ]:
q = q1b
print(q)
sql = llmu.chat_text_to_sql(gemini3pro, schema, q)
print(sql)


create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
the output should be presented as follows:
- format('{:5.2f}%', frac) for column frac
sorted by tyear;


SELECT 
    CAST(EXTRACT(YEAR FROM ts) AS INTEGER) AS year, 
    COUNT(*) AS noof_year, 
    (SELECT COUNT(*) FROM ccfraud) AS noof_all, 
    TO_CHAR(100.0 * COUNT(*) / (SELECT COUNT(*) FROM ccfraud), '90.00') || '%' AS frac 
FROM 
    ccfraud 
GROUP BY 
    year 
ORDER BY 
    year;



In [ ]:
ddbu.sql_print(con, sql)

## q2a

In [69]:
q = q2a
print(q)
sql = llmu.chat_text_to_sql(gemini3pro, schema, q)
print(sql)


create a SQL query that outputs the following columns:
- trx_time_delta_bin time difference between current and previous transaction per userid either 'less_5min' or 'more_eq_5min'
- noof_trx number of records per bin
- noof_fraud number of fraud records per bin
- frac fraction of fraud records per bin
sorted by userid, trx_time_delta_bin;


WITH TransactionDeltas AS (
    SELECT 
        userid,
        is_fraud,
        ts - LAG(ts) OVER (PARTITION BY userid ORDER BY ts) AS time_diff
    FROM ccfraud
)
SELECT 
    userid,
    CASE 
        WHEN time_diff < INTERVAL '5 minutes' THEN 'less_5min'
        ELSE 'more_eq_5min'
    END AS trx_time_delta_bin,
    COUNT(*) AS noof_trx,
    SUM(CASE WHEN is_fraud = 'Yes' THEN 1 ELSE 0 END) AS noof_fraud,
    CAST(SUM(CASE WHEN is_fraud = 'Yes' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS frac
FROM TransactionDeltas
WHERE time_diff IS NOT NULL
GROUP BY userid, trx_time_delta_bin
ORDER BY userid, trx_time_delta_bin;



In [70]:
ddbu.sql_print(con, sql)

┌────────┬────────────────────┬──────────┬────────────┬───────────────┐
│ userid │ trx_time_delta_bin │ noof_trx │ noof_fraud │     frac      │
│ int16  │      varchar       │  int64   │   int128   │     float     │
├────────┼────────────────────┼──────────┼────────────┼───────────────┤
│      0 │ less_5min          │      510 │          0 │           0.0 │
│      0 │ more_eq_5min       │    11051 │         17 │  0.0015383223 │
│      1 │ less_5min          │      214 │          1 │   0.004672897 │
│      1 │ more_eq_5min       │     5446 │         29 │   0.005325009 │
│      2 │ less_5min          │      777 │          6 │  0.0077220076 │
│      2 │ more_eq_5min       │    25520 │         34 │  0.0013322884 │
│      3 │ less_5min          │      401 │          2 │  0.0049875313 │
│      3 │ more_eq_5min       │     8421 │         18 │  0.0021375134 │
│      4 │ less_5min          │      191 │          0 │           0.0 │
│      4 │ more_eq_5min       │     9183 │          1 │ 0.000108

# GPT5MINI

In [18]:
gpt5mini = llmu.stats_c(llmu.GPT5MINI)

## q1

In [25]:
q = q1
print(f"q1:{q}")
sql = llmu.chat_text_to_sql(gpt5mini, schema, q)
print(sql)

q1:
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;

SELECT
  EXTRACT(YEAR FROM ts)::INT AS year,
  COUNT(*) AS noof_year,
  SUM(COUNT(*)) OVER () AS noof_all,
  ROUND(COUNT(*)::NUMERIC / SUM(COUNT(*)) OVER (), 6) AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY EXTRACT(YEAR FROM ts);


In [26]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬──────────┐
│ year  │ noof_year │ noof_all │   frac   │
│ int32 │   int64   │  int128  │  double  │
├───────┼───────────┼──────────┼──────────┤
│  2010 │   1491225 │ 16575073 │ 0.089968 │
│  2011 │   1570551 │ 16575073 │ 0.094754 │
│  2012 │   1610829 │ 16575073 │ 0.097184 │
│  2013 │   1650917 │ 16575073 │ 0.099602 │
│  2014 │   1672343 │ 16575073 │ 0.100895 │
│  2015 │   1701371 │ 16575073 │ 0.102646 │
│  2016 │   1708924 │ 16575073 │ 0.103102 │
│  2017 │   1723360 │ 16575073 │ 0.103973 │
│  2018 │   1721615 │ 16575073 │ 0.103868 │
│  2019 │   1723938 │ 16575073 │ 0.104008 │
├───────┴───────────┴──────────┴──────────┤
│ 10 rows                       4 columns │
└─────────────────────────────────────────┘



## q2

In [27]:
q = q2
print(f"q:{q}")
sql = llmu.chat_text_to_sql(gpt5mini, schema, q)
print(sql)

q:
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
the output should be presented as follows:
- format('{:5.2f}%', frac) for column frac
sorted by tyear;

SELECT
  EXTRACT(YEAR FROM ts)::INT AS tyear,
  COUNT(*) AS noof_year,
  tot.total_count AS noof_all,
  format('%5.2f%%', (COUNT(*)::numeric / tot.total_count::numeric) * 100) AS frac
FROM ccfraud
CROSS JOIN (SELECT COUNT(*) AS total_count FROM ccfraud) AS tot
GROUP BY tyear, tot.total_count
ORDER BY tyear;


In [28]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────┐
│ tyear │ noof_year │ noof_all │  frac   │
│ int32 │   int64   │  int64   │ varchar │
├───────┼───────────┼──────────┼─────────┤
│  2010 │   1491225 │ 16575073 │ %5.2f%% │
│  2011 │   1570551 │ 16575073 │ %5.2f%% │
│  2012 │   1610829 │ 16575073 │ %5.2f%% │
│  2013 │   1650917 │ 16575073 │ %5.2f%% │
│  2014 │   1672343 │ 16575073 │ %5.2f%% │
│  2015 │   1701371 │ 16575073 │ %5.2f%% │
│  2016 │   1708924 │ 16575073 │ %5.2f%% │
│  2017 │   1723360 │ 16575073 │ %5.2f%% │
│  2018 │   1721615 │ 16575073 │ %5.2f%% │
│  2019 │   1723938 │ 16575073 │ %5.2f%% │
├───────┴───────────┴──────────┴─────────┤
│ 10 rows                      4 columns │
└────────────────────────────────────────┘



## q3

In [19]:
q = q3
print(f"q:{q}")
sql = llmu.chat_text_to_sql(gpt5mini, schema, q)
print(sql)

q:
create a SQL query that outputs the following columns:
- trx_time_delta_bin time difference between current and previous transaction per userid either 'less_5min' or 'more_eq_5min'
- noof_trx number of records per bin
- noof_fraud number of fraud records per bin
- frac fraction of fraud records per bin
regard the following:
- only consider bins where noof_fraud > 0
sorted by userid, trx_time_delta_bin;

WITH deltas AS (
  SELECT
    userid,
    ts,
    lag(ts) OVER (PARTITION BY userid ORDER BY ts) AS prev_ts,
    is_fraud
  FROM ccfraud
)
SELECT
  userid,
  CASE WHEN EXTRACT(EPOCH FROM (ts - prev_ts)) < 300 THEN 'less_5min' ELSE 'more_eq_5min' END AS trx_time_delta_bin,
  COUNT(*) AS noof_trx,
  SUM(CASE WHEN lower(coalesce(is_fraud, '')) IN ('1','t','true','y','yes') THEN 1 ELSE 0 END) AS noof_fraud,
  SUM(CASE WHEN lower(coalesce(is_fraud, '')) IN ('1','t','true','y','yes') THEN 1 ELSE 0 END)::float / COUNT(*) AS frac
FROM deltas
WHERE prev_ts IS NOT NULL
GROUP BY userid, CASE WH

In [20]:
ddbu.sql_print(con, sql)

┌────────┬────────────────────┬──────────┬────────────┬───────────────┐
│ userid │ trx_time_delta_bin │ noof_trx │ noof_fraud │     frac      │
│ int16  │      varchar       │  int64   │   int128   │     float     │
├────────┼────────────────────┼──────────┼────────────┼───────────────┤
│      0 │ more_eq_5min       │    11051 │         17 │  0.0015383223 │
│      1 │ less_5min          │      214 │          1 │   0.004672897 │
│      1 │ more_eq_5min       │     5446 │         29 │   0.005325009 │
│      2 │ less_5min          │      777 │          6 │  0.0077220076 │
│      2 │ more_eq_5min       │    25520 │         34 │  0.0013322884 │
│      3 │ less_5min          │      401 │          2 │  0.0049875313 │
│      3 │ more_eq_5min       │     8421 │         18 │  0.0021375134 │
│      4 │ more_eq_5min       │     9183 │          1 │ 0.00010889687 │
│      5 │ less_5min          │      255 │          2 │   0.007843138 │
│      5 │ more_eq_5min       │    11863 │         11 │  0.00092

# OPUS41

In [19]:
opus41 = llmu.stats_c(llmu.OPUS41)

## q1

In [20]:
q = q1
print(f"q1:{q}")
sql = llmu.chat_text_to_sql(opus41, schema, q)
print(sql)

q1:
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
sorted by tyear;


SELECT 
    EXTRACT(YEAR FROM ts) AS year,
    COUNT(*) AS noof_year,
    SUM(COUNT(*)) OVER () AS noof_all,
    COUNT(*)::FLOAT / SUM(COUNT(*)) OVER () AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY year;



In [21]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────────┐
│ year  │ noof_year │ noof_all │    frac     │
│ int64 │   int64   │  int128  │    float    │
├───────┼───────────┼──────────┼─────────────┤
│  2010 │   1491225 │ 16575073 │  0.08996793 │
│  2011 │   1570551 │ 16575073 │ 0.094753794 │
│  2012 │   1610829 │ 16575073 │  0.09718382 │
│  2013 │   1650917 │ 16575073 │ 0.099602394 │
│  2014 │   1672343 │ 16575073 │  0.10089506 │
│  2015 │   1701371 │ 16575073 │ 0.102646366 │
│  2016 │   1708924 │ 16575073 │  0.10310205 │
│  2017 │   1723360 │ 16575073 │ 0.103972994 │
│  2018 │   1721615 │ 16575073 │  0.10386772 │
│  2019 │   1723938 │ 16575073 │  0.10400787 │
├───────┴───────────┴──────────┴─────────────┤
│ 10 rows                          4 columns │
└────────────────────────────────────────────┘



## q2

In [22]:
q = q2
print(f"q1:{q}")
sql = llmu.chat_text_to_sql(opus41, schema, q)
print(sql)

q1:
create a SQL query that outputs the following columns:
- year
- noof_year number of records per year
- noof_all total number of records
- frac fraction of records per year 
the output should be presented as follows:
- format('{:5.2f}%', frac) for column frac
sorted by tyear;


SELECT 
    EXTRACT(YEAR FROM ts) AS year,
    COUNT(*) AS noof_year,
    SUM(COUNT(*)) OVER () AS noof_all,
    FORMAT('%5.2f%%', (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ())) AS frac
FROM ccfraud
GROUP BY EXTRACT(YEAR FROM ts)
ORDER BY year;



In [23]:
ddbu.sql_print(con, sql)

┌───────┬───────────┬──────────┬─────────┐
│ year  │ noof_year │ noof_all │  frac   │
│ int64 │   int64   │  int128  │ varchar │
├───────┼───────────┼──────────┼─────────┤
│  2010 │   1491225 │ 16575073 │ %5.2f%% │
│  2011 │   1570551 │ 16575073 │ %5.2f%% │
│  2012 │   1610829 │ 16575073 │ %5.2f%% │
│  2013 │   1650917 │ 16575073 │ %5.2f%% │
│  2014 │   1672343 │ 16575073 │ %5.2f%% │
│  2015 │   1701371 │ 16575073 │ %5.2f%% │
│  2016 │   1708924 │ 16575073 │ %5.2f%% │
│  2017 │   1723360 │ 16575073 │ %5.2f%% │
│  2018 │   1721615 │ 16575073 │ %5.2f%% │
│  2019 │   1723938 │ 16575073 │ %5.2f%% │
├───────┴───────────┴──────────┴─────────┤
│ 10 rows                      4 columns │
└────────────────────────────────────────┘

